# Analysis - FiD cross attention score functions implementation
- FiD cross attention score


## CHECKING PARSER

In [1]:
import transformers
print(transformers.__version__)

3.0.2


In [2]:
# Accelerate transformer dependencies does not match with the Transformer dependencies
# from accelerate import Accelerator
# from accelerate.logging import get_logger
# from accelerate.utils import set_seed
import heapq
import pathlib
import shutil
from FiD.src.model import FiDT5
# from src.model import FiDEncoderForSequenceClassification

from pprint import pprint
from tqdm.auto import tqdm
from src.data import BinaryCustomDatasetShuffle
 
import json
import math
import os
import logging
import sys
# import evaluate
from util import utils
import pickle

import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset, DataLoader, SequentialSampler
from transformers import (
    AutoModelForSequenceClassification,
    AutoModel,
    AutoConfig,
    AutoTokenizer,
#     DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    set_seed,
#     get_scheduler,
)
# from util.arguments import ModelArguments, DataTrainingArguments, CustomTrainingArguments

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
# device = torch.device("cuda")

In [4]:
# device

In [5]:
# parser = HfArgumentParser((ModelArguments, DataTrainingArguments, CustomTrainingArguments))

# model_args, data_args, train_args = parser.parse_args_into_dataclasses([])

In [6]:
# pprint(vars(model_args))
# pprint(vars(data_args))
# pprint(vars(train_args))

# Get FiD Model

## modeling

In [7]:
from pprint import pprint
import numpy as np
import torch
from torch import nn
import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import T5PreTrainedModel
import copy
# from transformers.modeling_outputs import SequenceClassifierOutput
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

In [8]:
from FiD.src.model import FiDT5

In [9]:
model_path = '/scratch/philhoon-relevance/FiD/pretrained_models/nq_reader_large'
model = FiDT5.from_pretrained(model_path)
model.cuda()

FiDT5(
  (shared): Embedding(32128, 1024)
  (encoder): EncoderWrapper(
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): CheckpointWrapper(
          (module): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(in_features=1024, out_features=1024, bias=False)
                  (k): Linear(in_features=1024, out_features=1024, bias=False)
                  (v): Linear(in_features=1024, out_features=1024, bias=False)
                  (o): Linear(in_features=1024, out_features=1024, bias=False)
                  (relative_attention_bias): Embedding(32, 16)
                )
                (layer_norm): T5LayerNorm()
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (1): T5LayerFF(
                (DenseReluDense): T5DenseReluDense(
                  (wi): Linear(in_features=1024, out_fea

In [10]:
tokenizer = transformers.T5Tokenizer.from_pretrained('t5-base', return_dict=False)

In [11]:
from FiD.src import data

In [12]:
text_maxlength = 200
n_context = 2

In [13]:
collator_function = data.Collator(text_maxlength, tokenizer, n_context)

In [14]:
# example = {'id': 12, 
#            'question': "when was the public service commission original version of the upsc set up", 
#            'ctx': {"id": "17105334", 
#                    "title": "Bihar Public Service Commission", 
#                    "text": "3 of the Regulations, 1960 the Commission was constituted with a Chairman and 10 (ten) other members. The strength of members was reduced to 6 (six) after bifurcation of the State of Bihar and the State of Jharkhand vide notification no. 7/PSC-1013/95 (Part-3) Per 8262 dated 9 October 2002 of the Personnel & Administrative Reforms Department, Bihar. Article 320 and 321 of the Constitution of India prescribes the mandate of the State Public Service Commissions, which are: a)Recruitment by conduct of Competitive Examinations/ through interviews to the services of the State Government. b)Advising the State Government on the suitability of"}
#           }


In [15]:
eval_data = '/scratch/philhoon-relevance/FiD/open_domain_data/NQ_TEST_DPR_SELECTION/strict_positive_naive_damaging_remove_damage_irrelevant_relevant.json'

In [16]:
eval_examples = data.load_data(
    eval_data, 
)

## Testing 3 samples

In [17]:
eval_examples = eval_examples[:3]

In [18]:
for eg in eval_examples:
    print(f"ID : {eg['id']}")
    print(f"Question : {eg['question']}")
    print(f"# of ctxs : {len(eg['ctxs'])}")
    

ID : 1
Question : who got the first nobel prize in physics
# of ctxs : 1
ID : 2
Question : when is the next deadpool movie being released
# of ctxs : 1
ID : 3
Question : which mode is used for short wave broadcast service
# of ctxs : 100


In [19]:
eval_dataset = data.Dataset(
    eval_examples, 
    2, 
)

In [20]:
for eg in eval_dataset:
    print(f"index : {eg['index']}")
    print(f"question : {eg['question']}")
    print(f"# passages : {len(eg['passages'])}")

index : 0
question : question: who got the first nobel prize in physics
# passages : 1
index : 1
question : question: when is the next deadpool movie being released
# passages : 1
index : 2
question : question: which mode is used for short wave broadcast service
# passages : 2


In [21]:
# eval_dataset[2]

In [22]:
per_gpu_batch_size = 3

In [23]:
eval_sampler = SequentialSampler(eval_dataset) 
eval_dataloader = DataLoader(
    eval_dataset, 
    sampler=eval_sampler, 
    batch_size=per_gpu_batch_size,
    num_workers=10,
    collate_fn=collator_function
)

In [24]:
# eval_iter = iter(eval_dataloader)

In [25]:
model.eval()
# if hasattr(model, "module")

FiDT5(
  (shared): Embedding(32128, 1024)
  (encoder): EncoderWrapper(
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): CheckpointWrapper(
          (module): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(in_features=1024, out_features=1024, bias=False)
                  (k): Linear(in_features=1024, out_features=1024, bias=False)
                  (v): Linear(in_features=1024, out_features=1024, bias=False)
                  (o): Linear(in_features=1024, out_features=1024, bias=False)
                  (relative_attention_bias): Embedding(32, 16)
                )
                (layer_norm): T5LayerNorm()
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (1): T5LayerFF(
                (DenseReluDense): T5DenseReluDense(
                  (wi): Linear(in_features=1024, out_fea

## overwrite_forward_crossattention

```python
def overwrite_forward_crossattention(self):
    """
    Replace cross-attention forward function, only used to save
    cross-attention scores.
    """
    for mod in self.decoder.block:
        attn = mod.layer[1].EncDecAttention
        
        ## overwrite forward function of EncDecAttention 
        ## Same as using functool.partial
        attn.forward = types.MethodType(a, attn)
        
```

## reset_score_storage
```python
def reset_score_storage(self):
    """
    Reset score storage, only used when cross-attention scores are saved
    to train a retriever.
    """
    for mod in self.decoder.block:
        mod.layer[1].EncDecAttention.score_storage = None
```

In [26]:
# # 24 decoder layers
# decoder_layers = 0
# for mod in model.decoder.block:
#     decoder_layers += 1
#     # This is EncDecAttention Layer
#     # attn = mod.layer[1].EncDecAttention
# print(f'# of decoder_layers : {decoder_layers}')

## Original 

In [27]:
# model.overwrite_forward_crossattention()
# model.reset_score_storage() 

## Implemented

In [28]:
model.overwrite_forward_crossattention_token()
model.reset_score_storage_token() 
model.reset_score_storage() 

In [29]:
iter_object = iter(eval_dataloader)

## get_crossattention_scores

```python
    def get_crossattention_scores(self, context_mask):
        """
        Cross-attention scores are aggregated to obtain a single scalar per
        passage. This scalar can be seen as a similarity score between the
        question and the input passage. It is obtained by averaging the
        cross-attention scores obtained on the first decoded token over heads,
        layers, and tokens of the input passage.

        More details in Distilling Knowledge from Reader to Retriever:
        https://arxiv.org/abs/2012.04584.
        """
        scores = []
        n_passages = context_mask.size(1)
        for mod in self.decoder.block:
            scores.append(mod.layer[1].EncDecAttention.score_storage)
        scores = torch.cat(scores, dim=2)
        bsz, n_heads, n_layers, _ = scores.size()
        # batch_size, n_head, n_layers, n_passages, text_maxlength
        scores = scores.view(bsz, n_heads, n_layers, n_passages, -1)
        scores = scores.masked_fill(~context_mask[:, None, None], 0.)
        scores = scores.sum(dim=[1, 2, 4])
        ntokens = context_mask.sum(dim=[2]) * n_layers * n_heads
        scores = scores/ntokens
        return scores

```

In [30]:
batch = next(iter_object)

In [31]:
(idx, _, _, context_ids, context_mask) = batch

In [32]:
idx

tensor([0, 1, 2])

In [33]:
print(context_ids.shape)
print(context_mask.shape)

torch.Size([3, 2, 200])
torch.Size([3, 2, 200])


In [34]:
# model.reset_score_storage()

In [35]:
with torch.no_grad():
    outputs = model.generate(
                input_ids=context_ids.cuda(),
                attention_mask=context_mask.cuda(),
                max_length=50,
            )

In [36]:
crossattention_scores = model.get_crossattention_scores(context_mask.cuda())

In [37]:
crossattention_scores.shape

torch.Size([3, 2])

In [38]:
print(outputs.shape)

torch.Size([3, 9])


In [39]:
for k, o in enumerate(outputs):
    print(tokenizer.convert_ids_to_tokens(o))
    ans = tokenizer.decode(o, skip_special_tokens=True)
    print(k)
    print(o.shape)
    print(ans)

['<pad>', '▁Wilhelm', '▁Con', 'rad', '▁Rö', 'n', 't', 'gen', '</s>']
0
torch.Size([9])
Wilhelm Conrad Röntgen
['<pad>', '▁May', '▁18,', '▁2018', '</s>', '<pad>', '<pad>', '<pad>', '<pad>']
1
torch.Size([9])
May 18, 2018
['<pad>', '▁', 'ampli', 'tude', '▁modul', 'ation', '</s>', '<pad>', '<pad>']
2
torch.Size([9])
amplitude modulation


In [40]:
context_ids.size(1)

2

In [41]:
scores = []
n_passages = context_mask.size(1)
for mod in model.decoder.block:
    scores.append(mod.layer[1].EncDecAttention.score_storage)

In [42]:
for s in scores:
    print(s.shape)

torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])
torch.Size([3, 16, 1, 400])


In [43]:
test_score = torch.cat(scores, dim=2)

In [44]:
test_score.shape

torch.Size([3, 16, 24, 400])

In [45]:
bsz, n_heads, n_layers, _ = test_score.size()

In [48]:
context_mask = context_mask.to(device='cuda')
print(context_mask.is_cuda)

True


In [49]:
test_score = test_score.view(bsz, n_heads, n_layers, n_passages, -1)
test_score = test_score.masked_fill(~context_mask[:, None, None], 0.)
test_score = test_score.sum(dim=[1, 2, 4])

In [50]:
print(test_score.shape)

torch.Size([3, 2])


In [51]:
ntokens = context_mask.sum(dim=[2]) * n_layers * n_heads

In [52]:
ntokens

tensor([[71808,     0],
        [68352,     0],
        [58368, 60672]], device='cuda:0')

In [53]:
test_score = test_score/ntokens

In [54]:
test_score.shape

torch.Size([3, 2])

In [55]:
print(context_mask.shape)

torch.Size([3, 2, 200])


### get_crossattention_scores_token


```python
    def get_crossattention_scores(self, context_mask):
        """
        Cross-attention scores are aggregated to obtain a single scalar per
        passage. This scalar can be seen as a similarity score between the
        question and the input passage. It is obtained by averaging the
        cross-attention scores obtained on the first decoded token over heads,
        layers, and tokens of the input passage.

        More details in Distilling Knowledge from Reader to Retriever:
        https://arxiv.org/abs/2012.04584.
        """
        scores = []
        n_passages = context_mask.size(1)
        for mod in self.decoder.block:
            scores.append(mod.layer[1].EncDecAttention.score_storage)
        scores = torch.cat(scores, dim=2)
        bsz, n_heads, n_layers, _ = scores.size()
        # batch_size, n_head, n_layers, n_passages, text_maxlength
        scores = scores.view(bsz, n_heads, n_layers, n_passages, -1)
        scores = scores.masked_fill(~context_mask[:, None, None], 0.)
        scores = scores.sum(dim=[1, 2, 4])
        ntokens = context_mask.sum(dim=[2]) * n_layers * n_heads
        scores = scores/ntokens
        return scores

```

In [56]:
scores_token_lst = []
for mod in model.decoder.block:
    att_score_layer_lst = mod.layer[1].EncDecAttention.score_storage_token
    att_score_layer = torch.cat(att_score_layer_lst, dim=2)
    scores_token_lst.append(att_score_layer)
att_score_by_token = torch.stack(scores_token_lst, dim = 0)
print(att_score_by_token.shape)
# Layers, batch_size, n_heads, decode_tokens, n_passages * text_maxlength 

torch.Size([24, 3, 16, 8, 400])


In [57]:
# n_layers, batch_size, n_heads, decode_tokens, n_passages * text_maxlength 
print(att_score_by_token.shape)

torch.Size([24, 3, 16, 8, 400])


In [58]:
n_layers, bsz, n_head, n_dec_tokens, _ = att_score_by_token.size()

In [59]:
print(n_layers, bsz, n_head, n_dec_tokens, _)

24 3 16 8 400


###  Permute Axis 1
     - from : n_layers, batch_size, n_heads, decode_tokens, n_passages * text_maxlength 
     - to : batch_size, decode_tokens, n_heads, n_layers, n_passages, input_max_length

In [60]:
att_score_by_token = att_score_by_token.permute(1, 3, 2, 0, 4)

In [61]:
print(att_score_by_token.shape)

torch.Size([3, 8, 16, 24, 400])


In [62]:
att_score_by_token = att_score_by_token.view(bsz, n_dec_tokens, n_head, n_layers, n_passages, -1)

In [63]:
print(att_score_by_token.shape)

torch.Size([3, 8, 16, 24, 2, 200])


In [64]:
att_score_by_token = att_score_by_token.masked_fill(~context_mask[:, None, None, None], 0.)

### Check validity of att_score_by_token

In [65]:
att_score_by_token[2,-1,-1, -1,:]

tensor([[ 0.4089,  0.0537,  0.6166,  1.4100, -0.6440, -0.5910,  0.6947,  0.8202,
          1.1297, -0.8426, -2.8562,  0.2782,  0.5422, -2.3797, -1.2527, -1.3274,
         -3.7673, -0.7114, -2.6371, -1.8213, -1.2492, -1.6385, -1.4740, -0.0855,
         -1.6259,  1.3126, -5.7921, -2.5179, -1.2961, -2.3236, -3.5175, -2.5686,
         -0.7658, -0.3719, -0.4020, -1.1009, -0.9070, -1.2886,  0.4926,  0.3916,
          1.9654,  0.3270,  0.7716, -0.2830, -1.4503, -1.4470, -0.0594,  1.1424,
          0.6825,  3.0061, -1.1830, -1.3062, -2.0726, -2.1800, -0.5330,  0.0904,
         -1.4817, -0.2283, -1.9703, -1.1346, -2.7439, -0.7345,  0.5759, -1.3072,
         -2.3590,  0.0672, -1.6369,  2.6603, -0.3806, -3.3741, -1.9084, -2.1624,
         -2.8405, -3.1403, -2.5993, -0.7304, -0.0198, -1.7688, -2.0961,  2.9047,
         -2.8840, -2.0174,  1.0311,  2.7313,  0.0604,  0.6807, -1.8110, -2.6885,
         -2.7305, -1.4246, -2.5593,  3.2859, -2.2351, -0.5972, -0.6372, -1.9197,
          3.0936, -3.2779, -

###  Permute Axis 2
     - from : batch_size, decode_tokens, n_heads, n_layers, n_passages, input_max_length
     - to : batch_size, n_passages, decode_tokens, n_heads, n_layers, input_max_length

In [66]:
print(att_score_by_token.shape)
att_score_by_token = att_score_by_token.permute(0, 4, 1, 2, 3, 5)
print(att_score_by_token.shape)

torch.Size([3, 8, 16, 24, 2, 200])
torch.Size([3, 2, 8, 16, 24, 200])


## Average across the Heads

In [124]:
att_score_by_token.shape

torch.Size([3, 2, 8, 16, 24, 200])

In [129]:
att_score_by_token.shape

torch.Size([3, 2, 8, 16, 24, 200])

In [127]:
ave_att = torch.mean(att_score_by_token, 3)

In [128]:
ave_att.shape

torch.Size([3, 2, 8, 24, 200])

In [135]:
layer_layer = ave_att[:,:,:,-1,:]

In [136]:
layer_layer.shape

torch.Size([3, 2, 8, 200])

In [125]:
ntokens

tensor([[71808,     0],
        [68352,     0],
        [58368, 60672]], device='cuda:0')

In [67]:
test_lst = []

In [68]:
print(k)

2


In [69]:
for j in range(context_ids.size(1)):
    k_test = att_score_by_token[k, j]
    print(k_test.shape)

torch.Size([8, 16, 24, 200])
torch.Size([8, 16, 24, 200])


In [70]:
att_score_by_token.shape

torch.Size([3, 2, 8, 16, 24, 200])

In [71]:
att_score_by_token[2].shape

torch.Size([2, 8, 16, 24, 200])

## Testing Distributed Pickle file

In [99]:
att_score_lst = []

In [100]:
idx

tensor([0, 1, 2])

In [101]:
att_score_by_token.shape

torch.Size([3, 2, 8, 16, 24, 200])

In [102]:
att_score_by_token[0].is_cuda

True

In [103]:
for k, o in enumerate(outputs):
    print(idx[k])
    att_score_on_k = att_score_by_token[k].detach().cpu()
    print(att_score_on_k.shape)
    att_score_lst.append(
                        {
                            'id' : idx[k],
                            'attention_score': att_score_on_k,
                        }
                    )

tensor(0)
torch.Size([2, 8, 16, 24, 200])
tensor(1)
torch.Size([2, 8, 16, 24, 200])
tensor(2)
torch.Size([2, 8, 16, 24, 200])


In [104]:
len(att_score_lst)

3

In [107]:
att_score_lst[2]['attention_score']

tensor([[[[[ 7.4193e+00,  7.4041e+00, -1.0412e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [-5.5862e-01, -8.7128e-01, -2.2208e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [-3.7667e-01,  8.3930e-03, -1.3594e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           ...,
           [-5.9163e-01, -2.0779e+00, -1.9662e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [-1.0526e+00, -2.2059e+00, -2.5522e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 4.7861e-01, -4.9904e-01, -4.3489e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00]],

          [[ 7.3228e+00,  6.9019e+00, -3.1981e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 6.4861e-01,  6.5411e-01, -2.3689e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [-1.6608e+00, -1.8966e+00, -2.9741e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
 

In [ ]:
opt_name = 'opt_name'

In [ ]:
from pathlib import Path

In [ ]:
dir_path = Path('/home/philhoon/relevance_retrieval/test_pickle') / opt_name

In [ ]:
dir_path

In [ ]:
write_path = dir_path / 'tmp_dir'

In [ ]:
write_path

In [ ]:
write_path.mkdir(exist_ok=True)

In [ ]:
tmp_path = write_path / 'test-0.pickle'

In [ ]:
tmp_path

In [ ]:
import pickle

In [ ]:
with open(tmp_path, 'wb') as fw:
    pickle.dump(att_score_lst, fw)

In [ ]:
final_path = dir_path / 'attention_score.pickle'

In [ ]:
final_path

In [ ]:
glob_path = write_path / '*'

In [ ]:
glob_path

In [ ]:
results_path = write_path.glob('*.pickle')

In [ ]:
for i in results_path:
    print(i)

In [ ]:
alldata = []
for path in results_path:
    print(path)
    with open(path, 'rb') as f:
        data = pickle.load(f)
    print(len(data))
    alldata.extend(data)
    path.unlink()

In [ ]:
len(alldata)

In [ ]:
for ins in alldata:
    print(ins['id'])

In [ ]:
with open(final_path, 'wb') as fout:
    pickle.dump(alldata, fout)

In [ ]:
write_path.rmdir()

## Checking TEST output

In [108]:
pickle_file = '/data/philhoon-relevance/FiD/results/NQ_DPR/DEV_ATT_TOKEN/NQ_dev_1_context/attention_score.pickle'

In [109]:
with open(pickle_file, 'rb') as f:
    data = pickle.load(f)

In [110]:
len(data)

8757

In [115]:
data[100].keys()

dict_keys(['id', 'attention_score'])

In [117]:
data[100]['attention_score'].shape

torch.Size([1, 14, 16, 24, 200])

In [119]:
size_in_bytes = data[100]['attention_score'].nelement() * data[100]['attention_score'].element_size()

In [120]:
size_in_bytes

4300800

In [121]:
size_in_bytes/1024

4200.0

In [122]:
total_size = 0
for ins in data:
    size_ = ins['attention_score'].nelement() * ins['attention_score'].element_size()
    total_size += size_

In [123]:
print(total_size)

42174566400


In [ ]:
test_lst = []

In [ ]:
idx = [0,1,2]

In [ ]:
idx[2]

In [ ]:
test_lst.append(
    {
        'id' : idx[k],
        'attention_score' : att_score_by_token,
     
    }
)

In [ ]:
test_lst[0]['attention_score'].shape

In [ ]:
# Layers, batch_size, n_heads, , n_passages * text_maxlength 
# batch_size, decode_tokens, n_heads, Layers, n_passages * text_maxlength  
score_by_token = score_by_token.permute(1, 3, 2, 0, 4)

In [ ]:
# batch_size, decode_tokens, n_heads, Layers, n_passages * text_maxlength  
score_by_token.shape

In [ ]:
# batch_size, decode_tokens, n_heads, Layers, n_passages, text_maxlength 
# score_by_token = score_by_token.view(bsz, decoder_token, n_heads, n_layers, n_passages, -1)
score_by_token = score_by_token.view(1, 8, 16, 24, 2, -1)

In [ ]:
score_by_token.shape

In [ ]:
context_mask.shape

In [ ]:
context_mask[:, None, None, None].shape

In [ ]:
score_by_token = score_by_token.masked_fill(~context_mask[:, None, None, None], 0.)

In [ ]:
# batch_size, decoded_token, n_head, n_layers, n_passages, text_max_length
score_by_token.shape

In [ ]:
# sum across n_head, n_layers
score_by_token = score_by_token.sum(dim=[2, 3])

In [ ]:
# batch_size, decoded_token, n_passages, text_max_length
score_by_token.shape

In [ ]:
# average over n_head * n_layers
n_ = 16 * 24

In [ ]:
score_by_token = score_by_token/n_

In [ ]:
score_by_token.shape

In [ ]:
# BATCH, decode tokens, context, encoder tokens
for i in range(0,8):
    print(score_by_token[0,2,1,:].shape)
    print(score_by_token[0,2,1,:])

In [ ]:
context_mask[:,None, None].shape

In [ ]:
for s in scores:
    print(s.shape)
scores = torch.cat(scores, dim=2)

In [ ]:
# batch size, n_heads, n_layers, n_passages
scores.size()

In [ ]:
bsz, n_heads, n_layers, _ = scores.size()

In [ ]:
scores = scores.view(bsz, n_heads, n_layers, n_passages, -1)

In [ ]:
context_mask = context_mask.cuda()

In [ ]:
print(context_mask.is_cuda)
print(scores.is_cuda)

In [ ]:
scores.shape

In [ ]:
context_mask[:, None, None].shape

In [ ]:
scores = scores.masked_fill(~context_mask[:, None, None], 0.)

In [ ]:
# batch_size, n_heads, n_layers, n_passages, text_maxlength
scores.shape

In [ ]:
# sum across - n_heads, n_layers, text_maxlength
scores = scores.sum(dim=[1, 2, 4])

In [ ]:
# batch_size, n_passages
scores.shape

In [ ]:
# sum across =tokens
context_mask.shape

In [ ]:
context_mask.sum(dim=[2])

In [ ]:
ntokens = context_mask.sum(dim=[2]) * n_layers * n_heads

In [ ]:
ntokens.shape

In [ ]:
scores = scores/ntokens

In [ ]:
scores

In [ ]:
context_mask.is_cuda

In [ ]:
scores.shape

In [ ]:
test_ = scores[:, None, : , None]

In [ ]:
test_.shape

In [ ]:
scores_by_encoder_token_lst = []
 
for token_ind, s_ in enumerate(scores_token_lst, 0):
    print(f'decoded token {token_ind}')
    print(len(s_))
    print((s_[1].size()))
    scores = torch.cat(s_, dim=2)
    scores_by_encoder_token_lst.append(scores)

In [ ]:
scores_by_encoder_token_lst[0].shape

In [ ]:
len(scores_token_lst.shape)

In [ ]:
tokenizer.convert_ids_to_tokens([    0, 27545,  1193,  5672, 22087,    29,    17,   729,     1])

In [ ]:
# crossattention_scores = model.get_crossattention_scores(context_mask.cuda())

In [ ]:
# cross attention score on passage level
# crossattention_scores

In [ ]:
for k, o in enumerate(outputs):
    print(k)
    print(o.shape)
    print(o)
    ans = tokenizer.decode(o, skip_special_tokens=False)
#     print(tokenizer.decode([0,1], skip_special_tokens=False))
    print(ans)
#     ans = tokenizer.decode(o, skip_special_tokens=True)

In [ ]:
scores = []
n_passages = context_mask.size(1)

In [ ]:
scores = []
n_passages = context_mask.size(1)
for mod in self.decoder.block:
    scores.append(mod.layer[1].EncDecAttention.score_storage)

In [ ]:
# scores[0]

In [ ]:
cnt_scores = 0
for score in scores:
    cnt_scores += 1
    print(score.shape)
# batch_size, n_head, n_layers, n_passages, text_maxlength
print(cnt_scores)

In [ ]:
scores = torch.cat(scores, dim=2)

In [ ]:
# batch_size, n_head, n_layers, n_passages * text_maxlength ->  batch_size, n_head, n_layers, n_passages, text_maxlength
print(scores.shape)
scores = scores.view(1, 16, 24, 2, -1)
print(scores.shape)

In [ ]:
scores = scores 

In [ ]:
print(context_mask.shape)
print(context_mask[:, None, None].shape)

In [ ]:
context_mask = context_mask.cuda()

In [ ]:
~context_mask[:,None,None]

In [ ]:
scores = scores.masked_fill(~context_mask[:, None, None], 0.)

In [ ]:
scores.shape

In [ ]:
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        (idx, _, _, context_ids, context_mask) = batch

        model.reset_score_storage()

        outputs = model.generate(
            input_ids=context_ids.cuda(),
            attention_mask=context_mask.cuda(),
            max_length=50,
        )

        if opt.write_crossattention_scores:
            crossattention_scores = model.get_crossattention_scores(context_mask.cuda())

        for k, o in enumerate(outputs):
            ans = tokenizer.decode(o, skip_special_tokens=True)
            example = dataset.data[idx[k]]
            if 'answers' in example:
                score = src.evaluation.ems(ans, example['answers'])
                exactmatch.append(score)

            if opt.write_results:
                fw.write(str(example['id']) + "\t" + ans  + "\t" + str(score) + '\n')
            if opt.write_crossattention_scores:
                for j in range(context_ids.size(1)):
                    example['ctxs'][j]['score'] = crossattention_scores[k, j].item()

            total += 1
        if (i + 1) % opt.eval_print_freq == 0:
            log = f'Process rank:{opt.global_rank}, {i+1} / {len(dataloader)}'
            if len(exactmatch) == 0:
                log += '| no answer to compute scores'
            else:
                log += f' | average = {np.mean(exactmatch):.3f}'
            logger.warning(log)

In [ ]:
with torch.no_grad():

In [ ]:
type(attn)

In [ ]:
attn.forward = types.MethodType(cross_attention_forward, attn)

In [ ]:
# model.encoder => FiDT5.EncoderWrapper
# model.encoder.encoder => FiDT5.EncoderWrapper.encoder = T5 encoder Architecture w FiDT5 parameters
model_encoder = model.encoder.encoder

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained('t5-base', return_dict=False)

In [ ]:
output = tokenizer(example['question'])

In [ ]:
test_ids = np.expand_dims(np.array(output['input_ids']), 0)

In [ ]:
test_ids = torch.from_numpy(test_ids)
print(test_ids)

In [ ]:
test_attention = np.expand_dims(np.array(output['attention_mask']), 0)

In [ ]:
test_attention = torch.from_numpy(test_attention)
print(test_attention)

In [ ]:
result = model_encoder.forward(input_ids = test_ids, attention_mask = test_attention)

In [ ]:
vars(result)

In [ ]:
result['last_hidden_state'].size()

In [ ]:
config.num_labels

In [ ]:
class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.d_model, config.d_model)
        classifier_dropout = (
            config.dropout_rate
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.d_model, config.num_labels)

    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [ ]:
class AdaptivePooler(nn.Module):
    """ Calcualte weighted average of the inputs with learnable weights """
    def __init__(self, config):
        super().__init__()
        self.input_size = config.d_model
        self.w = nn.Linear(self.input_size, 1, bias=True)
        self.dropout = nn.Dropout(config.dropout_rate)
        self.classifier = nn.Linear(config.d_model, config.num_labels)

    def forward(self, inputs, mask=None):
        batch_size, seq_len, emb_dim = inputs.shape
        scores = torch.squeeze(self.w(inputs), dim=-1)
        weights = nn.functional.softmax(scores, dim=-1)
        if mask is not None:
            weights = weights * mask
            weights = weights / weights.sum(dim=-1, keepdims=True)
        outputs = (inputs.permute(2, 0, 1) * weights).sum(-1).T
        
        outputs = self.dropout(outputs)
        logits = self.classifier(outputs)
        
        return logits

In [ ]:
def init_weights_test(module):
    """ Initialize the weights """
    if isinstance(module, (nn.Linear, nn.Embedding)):
        module.weight.data.zero_()
       
    if isinstance(module, nn.Linear) and module.bias is not None:
        module.bias.data.zero_()

In [ ]:
adaptivePooler_.apply(init_weights_test)

In [ ]:
adaptivePooler_.w.weight

In [ ]:
config.problem_type

In [ ]:
class FiDEncoderForSequenceClassification(nn.Module):
    def __init__(self, config, model_encoder, pooler='adaptive'):
        super(FiDEncoderForSequenceClassification, self).__init__()
        self.num_labels = config.num_labels
        self.config = config
        
        self.encoder = model_encoder
        
        classification_class = AdaptivePooler if pooler == 'adaptive' else ClassificationHead
        self.classifier = classification_class(self.config)
        
        self.classifier.apply(self._init_weights)
        
    def _init_weights(self, module):
        """ Initialize the weights """
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=config.initializer_factor)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()
            
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        inputs_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        labels=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """

        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        logits = self.classifier(outputs[0], mask=attention_mask)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
encoder_classifier = FiDEncoderForSequenceClassification(config, model_encoder)

In [ ]:
outputs = encoder_classifier.forward(input_ids = test_ids, attention_mask = test_attention, return_dict = True)

In [ ]:
outputs

In [ ]:
no_decay = ["bias", "layer_norm.weight"]

In [ ]:
param = [p for n, p in encoder_classifier.named_parameters() if not any(nd in n for nd in no_decay)]

In [ ]:
param

In [ ]:
for n, p in model.named_parameters() :
    if any(nd in n for nd in no_decay):
        print(n)

# Checking Metric

In [ ]:
metric_acc = evaluate.load("accuracy")
metric_pre = evaluate.load('precision')
metric_re = evaluate.load('recall')
metric_f1 = evaluate.load('f1')

In [ ]:
predictions = np.array([1,1,1,1,1])
references = np.array([1,1,1,0,0])

In [ ]:
metric_acc.add_batch(
                predictions=predictions,
                references=references,
            )

In [ ]:
eval_metric = metric_acc.compute()
print(eval_metric)

In [ ]:
eval_metric = metric_acc.compute()

In [ ]:
eval_metric

In [ ]:
import types
import torch
import transformers
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss
import numpy as np

class FiDT5(transformers.T5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        self.wrap_encoder()

    def forward_(self, **kwargs):
        if 'input_ids' in kwargs:
            kwargs['input_ids'] = kwargs['input_ids'].view(kwargs['input_ids'].size(0), -1)
        if 'attention_mask' in kwargs:
            kwargs['attention_mask'] = kwargs['attention_mask'].view(kwargs['attention_mask'].size(0), -1)

        return super(FiDT5, self).forward(
            **kwargs
        )

    # We need to resize as B x (N * L) instead of (B * N) x L here
    # because the T5 forward method uses the input tensors to infer
    # dimensions used in the decoder.
    # EncoderWrapper resizes the inputs as (B * N) x L.
    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        if input_ids != None:
            # inputs might have already be resized in the generate method
            if input_ids.dim() == 3:
                self.encoder.n_passages = input_ids.size(1)
            input_ids = input_ids.view(input_ids.size(0), -1)
        if attention_mask != None:
            attention_mask = attention_mask.view(attention_mask.size(0), -1)
        return super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )

    # We need to resize the inputs here, as the generate method expect 2D tensors
    def generate(self, input_ids, attention_mask, max_length):
        self.encoder.n_passages = input_ids.size(1)
        return super().generate(
            input_ids=input_ids.view(input_ids.size(0), -1),
            attention_mask=attention_mask.view(attention_mask.size(0), -1),
            max_length=max_length
        )

    def wrap_encoder(self, use_checkpoint=False):
        """
        Wrap T5 encoder to obtain a Fusion-in-Decoder model.
        """
        self.encoder = EncoderWrapper(self.encoder, use_checkpoint=use_checkpoint)

    def unwrap_encoder(self):
        """
        Unwrap Fusion-in-Decoder encoder, useful to load T5 weights.
        """
        self.encoder = self.encoder.encoder
        block = []
        for mod in self.encoder.block:
            block.append(mod.module)
        block = nn.ModuleList(block)
        self.encoder.block = block

    def load_t5(self, state_dict):
        self.unwrap_encoder()
        self.load_state_dict(state_dict)
        self.wrap_encoder()

    def set_checkpoint(self, use_checkpoint):
        """
        Enable or disable checkpointing in the encoder.
        See https://pytorch.org/docs/stable/checkpoint.html
        """
        for mod in self.encoder.encoder.block:
            mod.use_checkpoint = use_checkpoint

    def reset_score_storage(self):
        """
        Reset score storage, only used when cross-attention scores are saved
        to train a retriever.
        """
        for mod in self.decoder.block:
            mod.layer[1].EncDecAttention.score_storage = None

    def get_crossattention_scores(self, context_mask):
        """
        Cross-attention scores are aggregated to obtain a single scalar per
        passage. This scalar can be seen as a similarity score between the
        question and the input passage. It is obtained by averaging the
        cross-attention scores obtained on the first decoded token over heads,
        layers, and tokens of the input passage.

        More details in Distilling Knowledge from Reader to Retriever:
        https://arxiv.org/abs/2012.04584.
        """
        scores = []
        n_passages = context_mask.size(1)
        for mod in self.decoder.block:
            scores.append(mod.layer[1].EncDecAttention.score_storage)
        scores = torch.cat(scores, dim=2)
        bsz, n_heads, n_layers, _ = scores.size()
        # batch_size, n_head, n_layers, n_passages, text_maxlength
        scores = scores.view(bsz, n_heads, n_layers, n_passages, -1)
        scores = scores.masked_fill(~context_mask[:, None, None], 0.)
        scores = scores.sum(dim=[1, 2, 4])
        ntokens = context_mask.sum(dim=[2]) * n_layers * n_heads
        scores = scores/ntokens
        return scores

    def overwrite_forward_crossattention(self):
        """
        Replace cross-attention forward function, only used to save
        cross-attention scores.
        """
        for mod in self.decoder.block:
            attn = mod.layer[1].EncDecAttention
            attn.forward = types.MethodType(cross_attention_forward, attn)

class EncoderWrapper(torch.nn.Module):
    """
    Encoder Wrapper for T5 Wrapper to obtain a Fusion-in-Decoder model.
    """
    def __init__(self, encoder, use_checkpoint=False):
        super().__init__()

        self.encoder = encoder
        apply_checkpoint_wrapper(self.encoder, use_checkpoint)

    def forward(self, input_ids=None, attention_mask=None, **kwargs,):
        # total_length = n_passages * passage_length
        bsz, total_length = input_ids.shape
        passage_length = total_length // self.n_passages
        input_ids = input_ids.view(bsz*self.n_passages, passage_length)
        attention_mask = attention_mask.view(bsz*self.n_passages, passage_length)
        outputs = self.encoder(input_ids, attention_mask, **kwargs)
        outputs = (outputs[0].view(bsz, self.n_passages*passage_length, -1), ) + outputs[1:]
        return outputs

class CheckpointWrapper(torch.nn.Module):
    """
    Wrapper replacing None outputs by empty tensors, which allows the use of
    checkpointing.
    """
    def __init__(self, module, use_checkpoint=False):
        super().__init__()
        self.module = module
        self.use_checkpoint = use_checkpoint

    def forward(self, hidden_states, attention_mask, position_bias, **kwargs):
        if self.use_checkpoint and self.training:
            kwargs = {k: v for k, v in kwargs.items() if v is not None}
            def custom_forward(*inputs):
                output = self.module(*inputs, **kwargs)
                empty = torch.tensor(
                    [],
                    dtype=torch.float,
                    device=output[0].device,
                    requires_grad=True)
                output = tuple(x if x is not None else empty for x in output)
                return output

            output = torch.utils.checkpoint.checkpoint(
                custom_forward,
                hidden_states,
                attention_mask,
                position_bias
            )
            output = tuple(x if x.size() != 0 else None for x in output)
        else:
            output = self.module(hidden_states, attention_mask, position_bias, **kwargs)
        return output

def apply_checkpoint_wrapper(t5stack, use_checkpoint):
    """
    Wrap each block of the encoder to enable checkpointing.
    """
    block = []
    for mod in t5stack.block:
        wrapped_mod = CheckpointWrapper(mod, use_checkpoint)
        block.append(wrapped_mod)
    block = nn.ModuleList(block)
    t5stack.block = block

In [ ]:
# !pip install evaluate

In [ ]:
from src.data import BinaryCustomDatasetShuffle

In [ ]:
from accelerate import Accelerator
from pprint import pprint
from tqdm import tqdm

In [ ]:
import json
import math
import os
import logging
import sys
import evaluate
from util import utils

import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForSequenceClassification, 
    AutoModel, 
    AutoConfig, 
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
    get_scheduler,
)
from util.arguments import ModelArguments, DataTrainingArguments 
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
class BinaryCustomDatasetShuffle(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length, shuffle = False):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = 'question: ' + self.instances[idx]['question'] + \
                 ' title: ' + self.instances[idx]['ctx']['title'] + \
                 ' context : ' + self.instances[idx]['ctx']['text']
        output = self.tokenizer(
            input_,
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length)

        item = {key: val for key, val in output.items()}
        # item['labels'] = torch.tensor(int(self.instances[idx]['em']))
        item['labels'] = int(self.instances[idx]['em'])

        return item

In [ ]:
report_to = 'wandb'
output_dir = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/scratch_test'

In [ ]:
# accelerator = Accelerator(log_with=args.report_to, logging_dir=args.output_dir) if args.with_tracking else Accelerator()
#     )

In [ ]:
accelerator = Accelerator(logging_dir=output_dir) 

In [ ]:
accelerator.device

In [ ]:
model_name_or_path = 'roberta-large'
num_labels = 2

In [ ]:
config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)

In [ ]:
pprint(config)

In [ ]:
print(config.num_labels)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
ignore_mismatched_sizes = True

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
        model_name_or_path,
        config=config,
        ignore_mismatched_sizes=ignore_mismatched_sizes,
    )

In [ ]:
train_file = '/data/philhoon-relevance/binary-classification/\
NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json'
eval_file = '/data/philhoon-relevance/binary-classification/\
NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json'

In [ ]:
train_data = utils.open_json(train_file)
eval_data = utils.open_json(eval_file)

In [ ]:
max_length = 200
shuffle = False

In [ ]:
train_dataset = BinaryCustomDatasetShuffle(train_data, tokenizer = tokenizer, \
                                           max_length = max_length, shuffle = shuffle)

In [ ]:
eval_dataset = BinaryCustomDatasetShuffle(eval_data, tokenizer = tokenizer, \
                                           max_length = max_length, shuffle = shuffle)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

In [ ]:
batch_size = 8

In [ ]:
train_dataloader = DataLoader(train_dataset,
                              shuffle = True,
                              collate_fn=data_collator,
                              batch_size=batch_size,
)

In [ ]:
eval_dataloader = DataLoader(eval_dataset,
                              shuffle = True,
                              collate_fn=data_collator,
                              batch_size=batch_size,
)

In [ ]:
no_decay = ["bias", "LayerNorm.weight"]

In [ ]:
weight_decay = 0.0

In [ ]:
optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]

In [ ]:
optimizer_grouped_parameters[0]["weight_decay"]

In [ ]:
learning_rate=5e-5

In [ ]:
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=learning_rate)

In [ ]:
lr_scheduler_type='linear'
num_warmup_steps = 0
# max_train_steps = 
num_train_epochs = 5
gradient_accumulation_steps = 1

In [ ]:
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)

In [ ]:
max_train_steps = num_train_epochs * num_update_steps_per_epoch

In [ ]:
max_train_steps

In [ ]:
lr_scheduler = get_scheduler(
        name=lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=max_train_steps,
)

In [ ]:
accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
    )

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
per_device_train_batch_size = 8

In [ ]:
total_batch_size = per_device_train_batch_size * accelerator.num_processes * gradient_accumulation_steps

In [ ]:
total_batch_size

In [ ]:
progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)

In [ ]:
num_train_epochs

In [ ]:
starting_epoch = 0
with_tracking = True

In [ ]:
checkpointing_steps = 50

In [ ]:
for epoch in range(starting_epoch, num_train_epochs):
    model.train()
    if with_tracking:
        total_loss = 0
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        
        if with_tracking:
            total_loss += loss.detach().float()
            
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        
        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            completed_steps += 1
            
        if isinstance(checkpointing_steps, int):
            if completed_steps % checkpointing_steps == 0:
                output_dir = f"step_{completed_steps }"
                if output_dir is not None:
                    output_dir = os.path.join(args.output_dir, output_dir)
                accelerator.save_state(output_dir)
        if completed_steps >= args.max_train_steps:
                break
                
                
    model.eval()
    samples_seen = 0
    for step, batch in enumerate(eval_dataloader):
         with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1) 
        predictions, references = accelerator.gather((predictions, batch["labels"]))
        
        if accelerator.num_processes > 1:
            if step == len(eval_dataloader) - 1:
                predictions = predictions[: len(eval_dataloader.dataset) - samples_seen]
                references = references[: len(eval_dataloader.dataset) - samples_seen]
            else:
                samples_seen += references.shape[0]
        
        metric.add_batch(
                predictions=predictions,
                references=references,
            )
        
        eval_metric = metric.compute()
        logger.info(f"epoch {epoch}: {eval_metric}")
        
        if args.with_tracking:
            accelerator.log(
                {
                    "accuracy" : eval_metric,
                    "train_loss": total_loss.item() / len(train_dataloader),
                    "epoch": epoch,
                    "step": completed_steps,
                },
                step=completed_steps,
            )


In [ ]:
metric_acc = evaluate.load("accuracy")
metric_pre = evaluate.load('precision')
metric_re = evaluate.load('recall')
metric_f1 = evaluate.load('f1')

In [ ]:
metric_acc

In [ ]:
accelerator.num_processes

In [ ]:
parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, TrainingArguments)
)
args = ["--model_name_or_path", 'allenai/longformer-large-4096', '--output_dir', './']
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)


In [ ]:
logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

In [ ]:
# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )


In [ ]:
# Set seed before initializing model.
set_seed(training_args.seed)

In [ ]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        num_labels=model_args.num_labels,
    )
tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    )
model = AutoModelForSequenceClassification.from_pretrained(
    model_args.model_name_or_path,
    config=config,
)

In [ ]:
if training_args.do_train:
    instances, cut_off, total_questions = preprocessing_data(
        data_args.train_file, 
        data_args.sample_size, 
        data_args.position)
    
    train_instance = instances[data_args.dev_size:]
    dev_instance = instances[:data_args.dev_size]
    
    train_dataset = CustomDataset(train_instance, 
                               tokenizer, 
                               model_args.max_seq_length)
    dev_dataset = CustomDataset(train_instance, 
                               tokenizer, 
                               model_args.max_seq_length)
    
    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

if training_args.do_eval:
    instances, cut_off, total_questions = preprocessing_data(
        data_args.test_file, 
        data_args.sample_size, 
        data_args.position)
    
    test_dataset = CustomDataset(instances, 
                               tokenizer, 
                               model_args.max_seq_length)
    
    


In [ ]:
# Get the metric function
metric = evaluate.load("xnli")

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)


In [ ]:
 # Initialize Trainer
data_collator = DataCollatorWithPadding(
    tokenizer, 
    pad_to_multiple_of=8,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_train else None,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=30)]
)

# Training
if training_args.do_train:
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    metrics = train_result.metrics
    max_train_samples = (
        data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
    )
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    trainer.save_model()  # Saves the tokenizer too for easy upload

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    
# Evaluation
if training_args.do_eval:
    logger.info("*** Evaluate ***")
    metrics = trainer.evaluate(eval_dataset=eval_dataset)

    max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
    metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

In [ ]:
metric

In [ ]:
data_args.dataset_name = a
    

In [ ]:
data.max_seq_length

In [ ]:
training_args.fp16

In [ ]:
bb

In [ ]:
def main():
    parser = HfArgumentParser(
        (ModelArguments, DataTrainingArguments, TrainingArguments)
    )
    
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    
    